In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import jet,RdYlGn

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load
import geopandas as gpd
import pandas as pd

# Define the bounding box for the entire data region using (Latitude, Longitude)
# This is the region of New York City that contains our temperature dataset
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

# Define the time window
# We will use a period of 3 months to search for data
time_window = "2021-06-01/2021-09-01"

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 50},"platform": {"in": ["landsat-8"]}},
)

items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))

# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 30  # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for crs=4326

data1 = stac_load(
    items,
    bands=["red", "green", "blue", "nir08"],
    crs="EPSG:2263", # Changed
    resolution=resolution,  # Meters
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

data2 = stac_load(
    items,
    bands=["lwir11"],
    crs="EPSG:2263", # Changed
    resolution=resolution,  # Meters
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

# Scale Factors for the RGB and NIR bands
scale1 = 0.0000275
offset1 = -0.2
data1 = data1.astype(float) * scale1 + offset1

# Scale Factors for the Surface Temperature band
scale2 = 0.00341802
offset2 = 149.0
kelvin_celsius = 273.15 # convert from Kelvin to Celsius
data2 = data2.astype(float) * scale2 + offset2 - kelvin_celsius

# Find the time slice closest to 2021-07-24
target_date = pd.to_datetime("2021-07-24")
time_diffs = abs(data2.time - target_date)
closest_time_index = time_diffs.argmin()

# Select the data for the closest time slice
lst_slice = data2.isel(time=closest_time_index)

# Save LST as Parquet
lst_df = lst_slice.to_dataframe(name='LST').reset_index()
gdf_lst = gpd.GeoDataFrame(lst_df, geometry=gpd.points_from_xy(lst_df.x, lst_df.y), crs="EPSG:2263")
gdf_lst = gdf_lst[['LST', 'geometry']] # Keep only necessary columns.
gdf_lst.to_parquet('../output/landsat_lst.parquet')